In [1]:
from scipy import spatial
exec(open("../database.py").read())
exec(open("../helperFunctions.py").read())
import numpy as np
from keras.datasets import mnist
from time import perf_counter
from ast import literal_eval

In [2]:
name = 'mnist-784-euclidean'
nameFull = name +'-true-labels.xlsx'
datasetTrainImages, datasetTestImages, _ = get_ann_benchmark_data(name)

datasetImages :  (70000, 784)
datasetLabels :  (70000,)


***Create scippy***

In [4]:
def createIndex(indexMethod, datasetImages):
    time_start = perf_counter()
    index = indexMethod(datasetImages)
    time_end = perf_counter()
    totalTime = (time_end - time_start)
    print(f'Building time {totalTime:.3f}')
    return (index, totalTime)
(indexedStruct, time) = createIndex(spatial.KDTree, datasetTrainImages)

Took 4.350 seconds


In [5]:
print(indexedStruct)

In [6]:
# (min, max) = createIndexNumerous(createIndex, NearestNeighbors, datasetImages, 10)
# print('min : ', min, '\n','max : ', max,)

In [15]:
indexes = []
distances = []
def measureTime(par, indexes, distances, datasetImages):
    totalTime = 0
    for i in range(par) : 
        xq = datasetImages[i:i+1].astype('float32') # Use the first image as the query vector
        time_start = perf_counter()
        distance, index = indexedStruct.query(xq, k=100, p=2, workers=1)
        time_end = perf_counter()
        totalTime += (time_end - time_start)
        indexes.append(index[0])
        distances.append(distance[0])
        # distances.append(np.sqrt(distance[0]))
        # indexes.append(index[0])
    # report the duration
    print(f'Searching time {totalTime:.3f}')
measureTime(1000, indexes, distances, datasetTestImages)

Took 89.570 seconds


In [ ]:
indexes = np.array(indexes)
distances = np.round(np.array(distances).astype(float), 4)

In [ ]:
print('indexes : ', indexes.shape)
print('distances : ', distances.shape)

In [ ]:
path = '../datasets/'+nameFull
(trueIndexes, trueDistances) = readDB(path)

In [ ]:
compareFirstTen(indexes, distances, trueIndexes, trueDistances)

In [ ]:
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.01)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.1)